### Imports

In [1]:
import sys, os

Alright, let's load the package and pick the `Project` since we want to start a project

In [2]:
from adaptivemd import Project

Let's open a project with a UNIQUE name. This will be the name used in the DB so make sure it is new and not too short. Opening a project will always create a non-existing project and reopen an exising one. You cannot chose between opening types as you would with a file. This is a precaution to not accidentally delete your project.

In [3]:
# Use this to completely remove the example-worker project from the database.
Project.delete('tutorial')

In [4]:
project = Project('tutorial')

Now we have a handle for our project. First thing is to set it up to work on a resource.

### Set the resource

What is a resource? A `Resource` specifies a shared filesystem with one or more clusteres attached to it. This can be your local machine or just a regular cluster or even a group of cluster that can access the same FS (like Titan, Eos and Rhea do).

Once you have chosen your place to store your results this way it is set for the project and can (at least should) not be altered since all file references are made to match this resource. Currently you can use the Fu Berlin Allegro Cluster or run locally. There are two specific local adaptations that include already the path to your conda installation. This simplifies the use of `openmm` or `pyemma`.

Let us pick a local resource on a laptop for now.

In [5]:
from adaptivemd import LocalCluster, AllegroCluster

first pick your resource -- where you want to run your simulation. Local or on Allegro

In [6]:
resource = LocalCluster()

Now you can add some additional paths, conda environment, etc, before we setup the project. This works by setting a special task `.wrapper` (see notebook 4 for more things to do)

In [7]:
resource.wrapper

In a nutshell, this dummy task has a `.pre` and `.post` list of commands you can add any command you want to be executed before every task you run.

In [8]:
resource.wrapper.pre.append('echo "Hello World"')

A task can also automatically add to the `PATH` variable, set environment variables and you can add conda environments

In [9]:
resource.wrapper.add_conda_env('my_env_python_27')

In [10]:
resource.wrapper.add_path('/x/y/z')

In [11]:
resource.wrapper.environment['CONDA'] = 'True'

In [12]:
print resource.wrapper.description

Task: DummyTask
<pre>
export PATH=/x/y/z:$PATH
export CONDA=True
echo "Hello World"
</pre>
<main />
<post>
</post>


Let's reset that now and just add a little comment

In [13]:
resource = LocalCluster()
resource.wrapper.pre.append('# This is part of the adaptivemd tutorial')

and finally initialize the project with this specific resource. This is done once for a project and should not be altered.

In [14]:
project.initialize(resource)

### Add `TaskGenerators`

TaskGenerators are instances whose purpose is to create tasks to be executed. This is similar to the
way Kernels work. A TaskGenerator will generate `Task` objects for you which will be translated into a `ComputeUnitDescription` and executed. In simple terms:

**The task generator creates the bash scripts for you that run a simulation or run pyemma.**

A task generator will be initialized with all parameters needed to make it work and it will now what needs to be staged to be used.

In [15]:
from adaptivemd.engine.openmm import OpenMMEngine
from adaptivemd.analysis.pyemma import PyEMMAAnalysis

from adaptivemd import File, Directory

#### The engine

A task generator that will create jobs to run simulations. Currently it uses a little python script that will excute OpenMM. It requires conda to be added to the PATH variable or at least openmm to be installed on the cluster. If you setup your resource correctly then this should all happen automatically.

First we define a `File` object. These are used to represent files anywhere, on the cluster or your local application. `File` like any complex object in adaptivemd can have a `.name` attribute that makes them easier to find later.

In [16]:
pdb_file = File('file://../files/alanine/alanine.pdb').named('initial_pdb').load()

Here we used a special prefix that can point to specific locations. 

- `file://` points to files on your local machine. 
- `worker://` specifies files on the current working directory of the executing node. Usually these are temprary files for a single execution.
- `shared://` specifies the root shared FS directory (e.g. `NO_BACKUP/` on Allegro) Use this to import and export files that are already on the cluster.
- `staging://` a special scheduler specific directory where files are moved after they are completed on a node and should be used for later. Use this to relate to files that should be stored or reused. After you one excution is done you usually move all important files to this place.
- `sandbox://` this should not concern you and is a special RP folder where all pilot/session folders are located.

The `.load()` at the end is important. It causes the `File` object to load the content of the file and if you save the `File` object, the actual file is stored with it. This way it can simply be rewritten on the cluster or anywhere else.

So let's do an example for an OpenMM engine. This is simply a small python script that makes OpenMM look like a executable. It run a simulation by providing an initial frame, OpenMM specific system.xml and integrator.xml files and some additional parameters like the platform name, how often to store simulation frames, etc.

In [17]:
engine = OpenMMEngine(
    pdb_file=pdb_file,
    system_file=File('file://../files/alanine/system.xml').load(),
    integrator_file=File('file://../files/alanine/integrator.xml').load(),
    args='-r --report-interval 1 -p CPU'
).named('openmm')

We have now an OpenMMEngine which uses the previously made pdb `File` object and uses the location defined in there. The same for the OpenMM XML files and some args to run using the `CPU` kernel, etc.

Last we name the engine `openmm` to find it later.

In [18]:
engine.name

'openmm'

Next, we need to set the output types we want the engine to generate. We chose a stride of 10 for the `master` trajectory without selection and a second trajectory with only protein atoms and native stride.

Note that the stride and all frame number ALWAYS refer to the native steps used in the engine. In out example the engine uses `2fs` time steps. So master stores every `20fs` and protein every `2fs`

In [19]:
engine.add_output_type('master', 'master.dcd', stride=10)
engine.add_output_type('protein', 'protein.dcd', stride=1, selection='protein')

#### The modeller

The instance to compute an MSM model of existing trajectories that you pass it. It is initialized with a `.pdb` file that is used to create features between the $c_\alpha$ atoms. This implementaton requires a PDB but in general this is not necessay. It is specific to my PyEMMAAnalysis show case.

In [20]:
modeller = PyEMMAAnalysis(
    engine=engine,
    outtype='protein',
    features={'add_inverse_distances': {'select_Backbone': None}}
).named('pyemma')

Again we name it `pyemma` for later reference.

The other two option chose which output type from the engine we want to analyse. We chose the protein trajectories since these are faster to load and have better time resolution.

The features dict expresses which features to use. In our case use all inverse distances between backbone c_alpha atoms.

#### Add generators to project

Next step is to add these to the project for later usage. We pick the `.generators` store and just add it. Consider a store to work like a `set()` in python. It contains objects only once and is not ordered. Therefore we need a name to find the objects later. Of course you can always iterate over all objects, but the order is not given.

To be precise there is an order in the time of creation of the object, but it is only accurate to seconds and it really is the time it was created and not stored.

In [21]:
project.generators.add(engine)
project.generators.add(modeller)

Note, that you cannot add the same engine twice. But if you create a new engine it will be considered different and hence you can store it again. 

In [34]:
import bson

In [47]:
c = project.storage.files._document.find()

In [48]:
f = next(c)

In [50]:
f['_']

[u'task',
 u'created',
 u'_file',
 u'_time',
 u'_cls',
 u'_id',
 u'_obj_uuid',
 u'_dict']

In [41]:
for f in c:
    print bson.BSON.encode(f)
    break

zR  _id %   cc1664de-0fd2-11e7-b14c-000000000004 
task created �n���4�A_file �"  import argparse
import ujson
from sys import stdout, exit
import socket
import numpy as np
import mdtraj as md

from simtk.openmm.app import *
from simtk.openmm import *
import simtk.unit as u

if __name__ == '__main__':

    # add further auto options here
    platform_properties = {
        'CUDA': ['Cuda_Device_Index', 'Cuda_Precision', 'Cuda_Use_Cpu_Pme',
                 'Cuda_Cuda_Compiler', 'Cuda_Temp_Directory', 'Cuda_Use_Blocking_Sync',
                 'Cuda_Deterministic_Forces'],
        'OpenCL': ['OpenCL_Device_Index', 'OpenCL_Precision', 'OpenCL_Use_Cpu_Pme',
                   'OpenCL_OpenCL_Platform_Index'],
        'CPU': ['CPU_Threads'],
        'Reference': []
    }

    platform_names = [
        Platform.getPlatform(no_platform).getName()
        for no_platform in range(Platform.getNumPlatforms())]

    parser = argparse.ArgumentParser(
        description='Run an MD simulation us

In [ ]:
print 

In [46]:
dir(f)

['ACTIVE_LONG',
 'CREATION_COUNT',
 'INSTANCE_UUID',
 '__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__eq__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__store__',
 '__str__',
 '__subclasshook__',
 '__time__',
 '__uuid__',
 '__weakref__',
 '_args',
 '_base',
 '_complete_target',
 '_exclude_private_attr',
 '_excluded_attr',
 '_file',
 '_find_by',
 '_ignore',
 '_included_attr',
 '_restore_name',
 '_restore_non_initial_attr',
 'allowed_drives',
 'args',
 'base',
 'base_cls',
 'base_cls_name',
 'basename',
 'basename_short',
 'clone',
 'cls',
 'copy',
 'create',
 'created',
 'default_drive',
 'descendants',
 'dirname',
 'drive',
 'exists',
 'extension',
 'from_dict',
 'generator',
 'get_file',
 'get_uuid',
 'has_file',
 'idx',
 'is_folder',
 'is_temp',
 'link',
 'load',
 'location',
 'modified',
 'move',
 'named',
 'objects',
 'path

In [45]:
for f in project.files:
    print hasattr(f, '_file_')
    print len(f._file) if f._file else ''
    print f.__dict__.keys()
#     print len(f._file_) if f._file else ''

False
8917
['location', 'resource', '__uuid__', '__store__', '__time__']
False
134220
['resource', 'name', 'location', '__time__', '__store__', '__uuid__']
False
165
['location', 'resource', '__uuid__', '__store__', '__time__']
False
320102
['location', 'resource', '__uuid__', '__store__', '__time__']
False
680
['location', 'resource', '__uuid__', '__store__', '__time__']


In [ ]:
f._file

### Create one intial trajectory

Finally we are ready to run a first trajectory that we will store as a point of reference in the project. Also it is nice to see how it works in general.

We are using a _Worker_ approach. This means simply that someone (in our case the user from inside a script or a notebook) creates a list of tasks to be done and some other instance (the worker) will actually do the work.

#### Create a `Trajectory` object

First we create the parameters for the engine to run the simulation. Since it seemed appropriate we use a `Trajectory` object (a special `File` with initial frame and length) as the input. You could of course pass these things separately, but this way, we can actualy reference the no yet existing trajectory and do stuff with it.

A Trajectory should have a unique name and so there is a project function to get you one. It uses numbers and makes sure that this number has not been used yet in the project.

In [22]:
trajectory = project.new_trajectory(engine['pdb_file'], 100, engine)
trajectory

Trajectory('alanine.pdb' >> [0..100])

This says, initial is `alanine.pdb` run for 100 frames and is named `xxxxxxxx.dcd`.

#### Create a `Task` object

Now, we want that this trajectory actually exists so we have to make it. This requires a `Task` object that _knows_ to describe a simulation. Since `Task` objects are very flexible and can be complex there are helper functions (i.e. factories) to get these in an easy manner, like the ones we already created just before. Let's use the openmm engine to create an openmm task now.

In [23]:
task = engine.run(trajectory)

As an alternative you can directly use the trajectory (which knows its engine) and call `.run()`

In [24]:
task = trajectory.run()

That's it, just take a trajectory description and turn it into a task that contains the shell commands and needed files, etc. 

#### Submit the task to the queue

Finally we need to add this task to the things we want to be done. This is easy and only requires saving the task to the project. This is done to the `project.tasks` bundle and once it has been stored it can be picked up by any worker to execute it.

In [25]:
project.queue(task)  # shortcut for project.tasks.add(task)

That is all we can do from here. To execute the tasks you need to run a worker using

```bash
adaptivemdworker -l tutorial --verbose
```

Once this is done, come back here and check your results. If you want you can execute the next cell which will block until the task has been completed.

In [26]:
print project.files
print project.trajectories

<StoredBundle for with 6 file(s) @ 0x112f1f390>
<ViewBundle for with 0 file(s) @ 0x112f1f690>


and close the project.

In [27]:
project.close()

The final project.close() will close the DB connection. 